# pandas Dataframe - Basic Operativity

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [2]:
from utilities.generators import generate_all
generate_all()

Generating p06_d2.txt...
Skipped p03_AAPL.csv (already existing)


Skipped p03_DAX.csv (already existing)
Skipped p07_d1.txt (already existing)
Skipped p01_d4.csv (already existing)
Skipped p01_volumes.txt (already existing)
Skipped p07_portfolio.h5 (already existing)
Skipped p06_d3.txt (already existing)
Skipped p01_d2.csv (already existing)
Skipped p01_prices.txt (already existing)
Skipped p07_d2.csv (already existing)
Skipped baby_names/ (already existing)
Skipped p01_d3.csv (already existing)


See [pandas documentation](<http://pandas.pydata.org/pandas-docs/stable/>) for more information and examples. Run the code at the end of the Notebook to generate the example files.

In [3]:
import numpy as np
import pandas as pd
from numpy import NaN
from addutils import side_by_side2
from addutils import css_notebook
from addutils import read_txt
from IPython.core.display import HTML
from faker import Factory
css_notebook()

## 1 File I/O and DataFrame Generation

Pandas reads and format data from many different file formats: `txt, csv, web, xls, mat`. In this case we use `read_csv` to read two textual data files.

First have a look to the file in its original form:

In [4]:
read_txt('temp/p01_prices.txt')

Date,AAPL,GOOG,JNJ,XOM

2015-03-03,129.36,573.64,102.34,87.62

2015-03-04,128.54,573.37,101.65,87.18

2015-03-05,126.41,575.33,102.52,86.74

2015-03-06,126.6,567.68,100.11,85.63



### 1.1 Create DataFrames with read_csv

This file can be read and formatted at the same time using `read_csv`. Lets read the two files `p01_prices.txt` and `p01_volumes.txt`

In [5]:
prices = pd.read_csv('temp/p01_prices.txt', index_col=0, parse_dates=[0])
volumes = pd.read_csv('temp/p01_volumes.txt', index_col=0, parse_dates=[0])

HTML(side_by_side2(prices, volumes))

Both "prices" and "volumes" datasets are 2D DataFrame objects:

In [6]:
type(prices)

pandas.core.frame.DataFrame

### 1.2 Create DataFrames from Python Dictionaries

In [7]:
fakeIT = Factory.create('it_IT')
data = {'Name' : [fakeIT.name() for i in xrange(5)],
        'Company' : [fakeIT.company() for i in xrange(5)],
        'City' : [fakeIT.city() for i in xrange(5)]}

In [8]:
df = pd.DataFrame(data, columns = ['Name','Company','City'])
df

,Name,Company,City
0,Bettino Rizzo,Rizzo s.r.l.,Artemide umbro
1,Dott. Soriana Palumbo,"Ferrara, Conte e Mancini s.r.l.",Baldassarre lido
2,Gianmarco Martino,"Gentile, Caputo e Parisi e figli",San Odone calabro
3,Sig. Vera Morelli,"Damico, Montanari e Giuliani s.r.l.",Esposito a mare
4,Moreno Silvestri,Monti-Coppola Group,Borgo Rebecca veneto


### 1.3 Create DataFrames from Items

In [9]:
df = pd.DataFrame.from_items([('Name', [fakeIT.name() for i in xrange(5)]),
                              ('Company', [fakeIT.company() for i in xrange(5)])])
df

,Name,Company
0,Sig. Diamante Caputo,Leone s.r.l.
1,Dott. Arturo Guerra,"Giuliani, Ruggiero e Silvestri Group"
2,Sig.ra Neri Donati,Gatti-Marini s.r.l.
3,Loretta De Angelis,Palumbo-Montanari SPA
4,Fortunata Sartori,Ruggiero-Grassi SPA


### 1.4 Create DataFrames fron Numpy Arrays

In [10]:
df = pd.DataFrame(np.array([[2,5],[3,6]]).T, index=list('ab'), columns=['ONE','TWO'])
df

,ONE,TWO
a,2,3
b,5,6


### 1.5 DataFrames can be converted in Numpy Arrays

In [11]:
np.asarray(df)

array([[2, 3],
       [5, 6]])

### 1.6 DataFrames, Series and Panels

In Pandas there are 3 main data structure types ("data container" objects):  

* **Series:**  for one-dimensional data 
* **DataFrames:**  for bi-dimensional data (matrices) 
* **Panels:** for 3D or nD data  

For simplicity, in this course we will describe only pandas Series and DataFrames.

## 2 Automatic Data Alignment

As you can see the dates has been interpreted correctly and used as row index. Notice that the rows in the two datafiles are misaligned, this is not a problem in pandas because the *Automatic Data Alignment* feature: an operation involving the two datasets will simply use `NaN` for the undefined (misaligned) values

In [12]:
prices*volumes

,AAPL,GOOG,JNJ,XOM
Date,,,,
2015-03-03,4.891917e+09,NaN,656992098,1182326756
2015-03-04,3.981874e+09,NaN,859471080,1169388930
2015-03-05,7.079541e+09,NaN,822292416,1085715906
2015-03-06,9.143786e+09,NaN,928850613,1368495845
2015-03-09,NaN,NaN,NaN,NaN
2015-03-10,NaN,NaN,NaN,NaN


Which can be better formatted to a "2 decimal places float number" with comma as thousands separator (see <a href="#options">Package Options</a>):

In [13]:
pd.set_option('display.float_format', lambda x: '{:,.1f}'.format(x))   # formatting
(prices*volumes).replace('nan', '-')    # replacing NaN

,AAPL,GOOG,JNJ,XOM
Date,,,,
2015-03-03,"4,891,916,568.0",-,"656,992,098.0","1,182,326,756.0"
2015-03-04,"3,981,873,558.0",-,"859,471,080.0","1,169,388,930.0"
2015-03-05,"7,079,541,486.0",-,"822,292,416.0","1,085,715,906.0"
2015-03-06,"9,143,786,280.0",-,"928,850,613.0","1,368,495,845.0"
2015-03-09,-,-,-,-
2015-03-10,-,-,-,-


Example: calculate the volume-weighted average price

In [14]:
vwap = (prices*volumes).sum()/volumes.sum()
vwap.dropna()

AAPL   127.4
JNJ    101.6
XOM     86.7
dtype: float64

## 3 Indexing

### 3.1 Label-Based Indexing

`.loc` is strictly label based, will raise KeyError when the items are not found, allowed inputs are:

* A single label
* A list or array of labels [’a’, ’b’, ’c’]
* A slice object with labels [’a’:’f’] (note that contrary to usual python slices, both the start and the stop are included!)

In [15]:
HTML(side_by_side2(prices, prices.loc['2012-11-21':'2012-11-27',['AAPL', 'GOOG']]))

KeyError: 'start bound [2012-11-21] is not the [index]'

Columns can be selected without specifying the index:

In [16]:
HTML(side_by_side2(prices, prices[['AAPL', 'GOOG']]))

### 3.2 Position-Based Indexing

`.iloc` is strictly position based, will raise KeyError when the items are out of bounds:

In [17]:
HTML(side_by_side2(prices, prices.iloc[1:5,[0, 1]]))

3.3 Mixed Indexing

### 3.3 Advanced Indexing - .ix

`.ix` will always **try first** to resolve **labeled** index (like `.loc`), **then** it will fall back on **potitional** indexing (like `.loc`).

Rows can be indexed using the ix method. Try by yourself:  

    prices.ix[1:4,0:2]                                # Position-based Indexing  
    prices.ix[:'2012-11-23']                          # Label-based Indexing on index (rows)
    prices.ix[:,[2,2,1]]                              # Duplicated values on columns 
    prices.ix[::2]                                    # One value every two rows
    prices.ix[::-1]                                   # Reverse rows
    prices.ix[prices['AAPL'] > 380]                   # Boolean indexing on index
    prices.ix[:,[len(c)<4 for c in prices.columns]]   # Boolean indexing on columns

In [18]:
prices.ix[:,[len(c)<4 for c in prices.columns]]

,JNJ,XOM
Date,,
2015-03-03,102.3,87.6
2015-03-04,101.7,87.2
2015-03-05,102.5,86.7
2015-03-06,100.1,85.6
2015-03-09,100.7,85.2
2015-03-10,99.5,84.3


## 4 DataFrame Basic Operations

### 4.1 Reindex/Reorder rows and columns

In [19]:
data = np.array([[2,5,8,11],[3,6,9,12]])
d1 = pd.DataFrame(data.T, index=list('abce'), columns=['K','W'])
HTML(side_by_side2(d1, pd.DataFrame(d1, index=list('baez'), columns=['W','K','T'])) )

### 4.2 Calculate new columns

In [20]:
d1['Z'] = d1['W']-d1['K']
d1['B'] = d1['W']>4
d1

,K,W,Z,B
a,2,3,1,False
b,5,6,1,True
c,8,9,1,True
e,11,12,1,True


### 4.3 Deleting rows and columns

In [21]:
d1['SUM'] = d1.sum(axis=1)
HTML(side_by_side2(d1, d1.drop(['b', 'c'], axis=0), d1.drop(['Z', 'B'], axis=1)))

### 4.4 Inserting colums in a specific position

In [22]:
d2 = d1.copy()  # .copy() method is needed to create a new object.
d2.insert(1, 'Exp(W)', np.exp(d1['W']))
HTML(side_by_side2(d1, d2, space=10))

Example: Indexing rows to create a new column with empty values, then use the Forward Fill Padding to fill the gaps

In [23]:
d1['part'] = d1['K'].ix[:2]
d1['part'] = d1['part'].fillna(method='ffill')
d1

,K,W,Z,B,SUM,part
a,2,3,1,False,6.0,2.0
b,5,6,1,True,13.0,5.0
c,8,9,1,True,19.0,5.0
e,11,12,1,True,25.0,5.0


### 4.5 Check if a value or a list of given values are contained in a specific column

In [24]:
# TODO: Upgrade side_by_side2 to include series
HTML(side_by_side2(d1, d1['K'].isin([3, 8])))


### 4.6 Rename columns

In [25]:
HTML(side_by_side2(d1, d1.rename(columns={'K':'ONE','W':'TWO','Z':'THREE'})))

,K,W,Z,B,SUM,part
a,2,3,1,False,6.0,2.0
b,5,6,1,True,13.0,5.0
c,8,9,1,True,19.0,5.0
e,11,12,1,True,25.0,5.0
,ONE,TWO,THREE,B,SUM,part
a,2,3,1,False,6.0,2.0
b,5,6,1,True,13.0,5.0
c,8,9,1,True,19.0,5.0
e,11,12,1,True,25.0,5.0


### 4.7 Iterate efficiently through rows

`iterrows` returns an iterator yielding each index value along with a Series containing the data in each row:

In [26]:
for row_index, row in d1.iterrows():
    print row_index, '**', ' - '.join([str(item) for item in row])

a ** 2 - 3 - 1 - False - 6.0 - 2.0
b ** 5 - 6 - 1 - True - 13.0 - 5.0
c ** 8 - 9 - 1 - True - 19.0 - 5.0
e ** 11 - 12 - 1 - True - 25.0 - 5.0


`itertuples` returns an iterator yielding a tuple for each row in the DataFrame. The first element of the tuple is the row’s corresponding index value, while the remaining elements are the row values:

In [27]:
for t in d1.itertuples():
    print t

('a', 2, 3, 1, False, 6.0, 2.0)
('b', 5, 6, 1, True, 13.0, 5.0)
('c', 8, 9, 1, True, 19.0, 5.0)
('e', 11, 12, 1, True, 25.0, 5.0)


## 5 Duplicated Data

### 5.1 Find duplicated data in columns

In [28]:
d3 = pd.read_csv('temp/p01_d2.csv', index_col=0)
d3['a dup'] = d3.duplicated(['a'])
d3['a+b dup'] = d3.duplicated(['a', 'b'])
d3['a+b dup - take last'] = d3.duplicated(['a', 'b'], take_last=True)
d3

,a,b,c,a dup,a+b dup,a+b dup - take last
0,one,x,-0.7,False,False,False
1,one,y,0.2,True,False,False
2,two,y,1.2,False,False,True
3,three,x,-1.2,False,False,False
4,two,y,0.7,True,True,False


### 5.2 Remove Duplicates

In [29]:
d3.drop_duplicates(['a', 'b'],take_last=True)

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


,a,b,c,a dup,a+b dup,a+b dup - take last
0,one,x,-0.7,False,False,False
1,one,y,0.2,True,False,False
3,three,x,-1.2,False,False,False
4,two,y,0.7,True,True,False


## 6 Working with Large Arrays

### 6.1 Control the DataFrame memory occupation

Let's start by generating a DataFrame from a Numpy Array. We'll see than there is no memory overhead on DataFrame Values:

In [30]:
rows, cols = 100, 100
np_array = np.array(np.random.randn(rows, cols), dtype=np.float64)
d4 = pd.DataFrame(np_array)
print 'Rows x Cols x 8:                     ', rows*cols*8
print 'np Array Memory Occupation:          ', np_array.nbytes
print 'Dataframe Values Memory Occupation:  ', d4.values.nbytes
print 'Dataframe Index Memory Occupation:   ', d4.index.nbytes
print 'Dataframe Columns Memory Occupation: ', d4.columns.nbytes

Rows x Cols x 8:                      80000
np Array Memory Occupation:           80000
Dataframe Values Memory Occupation:   80000
Dataframe Index Memory Occupation:    800
Dataframe Columns Memory Occupation:  800


To reduce the memory occupation it's possible to change the value's dtype:

In [31]:
d4 = d4.astype(dtype=np.float16)
print 'Dataframe Values Memory Occupation:  ', d4.values.nbytes

Dataframe Values Memory Occupation:   20000


If the data is sparse the Dataframe can be sparsified as well to save further resources with the `to_sparse()` method:

In [32]:
d4.ix[2:,4:] = np.nan
print 'Dataframe Values Memory Occupation:  ', d4.values.nbytes
d4 = d4.to_sparse()
print 'Dataframe Values Memory Occupation:  ', d4.values.nbytes

Dataframe Values Memory Occupation:   20000
Dataframe Values Memory Occupation:   80000


In this case rows and colums are np.int64 arrays:

In [33]:
d4.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], dtype='int64')

### 6.2 Explore large arrays

Working with large arrays: in Excel is difficult to explore arrays with thousands of lines and columns. Explore the pandas capabilities with the following code. The first line visualize the firts two lines, while the second actually load the whole file. Try to do the same in Excel for comparison.

In [34]:
d3 = pd.read_csv('example_data/p01_d3.csv.gz', compression='gzip')

In [35]:
for col in d3.columns:         
    print col, '-',

Istat - Comune - Provincia - Regione - Prefisso - CAP - CodFisco - Abitanti - Link -


Try by yourself:  

    d3.head()
    d3[d3.columns[:3]].head()
    d3[d3.columns[-4:-1]].tail()
    d3.ix[1000:1010, :7]
    d3.ix[:, 'Abitanti'].describe()

In [36]:
d3[d3.columns[-4:-1]].tail()

,CAP,CodFisco,Abitanti
8087,33020,M200,607
8088,13848,M201,1152
8089,87040,M202,2413
8090,83030,M203,1232
8091,89867,M204,2055


## 7 Column pct_change and shift

It is possibile to add multiple new columns to a `DataFrame`.

In [37]:
data = np.array([[3, 5, 7, 10, 13, 16, 56, 72],
                 [8, 16, 28, 37, 45, 57, 69, 90],
                 [3, 6, NaN, NaN, 15, 18, NaN, NaN],
                 [1, 2, 4, 7, 11, 16, 65, 88],
                 [NaN, NaN, NaN, NaN, 16, 19, 82, 91]])
d4 = pd.DataFrame(data.T, columns=['one', 'two', 'three', 'four', 'five'])
d4

,one,two,three,four,five
0,3.0,8.0,3.0,1.0,nan
1,5.0,16.0,6.0,2.0,nan
2,7.0,28.0,nan,4.0,nan
3,10.0,37.0,nan,7.0,nan
4,13.0,45.0,15.0,11.0,16.0
5,16.0,57.0,18.0,16.0,19.0
6,56.0,69.0,nan,65.0,82.0
7,72.0,90.0,nan,88.0,91.0


In [38]:
d4[['one ret','two ret']] = d4[['one','two']].pct_change()+1
d4

,one,two,three,four,five,one ret,two ret
0,3.0,8.0,3.0,1.0,nan,nan,nan
1,5.0,16.0,6.0,2.0,nan,1.7,2.0
2,7.0,28.0,nan,4.0,nan,1.4,1.8
3,10.0,37.0,nan,7.0,nan,1.4,1.3
4,13.0,45.0,15.0,11.0,16.0,1.3,1.2
5,16.0,57.0,18.0,16.0,19.0,1.2,1.3
6,56.0,69.0,nan,65.0,82.0,3.5,1.2
7,72.0,90.0,nan,88.0,91.0,1.3,1.3


In [39]:
d4['four var'] = np.log(d4['four'] - d4['four'].shift())
d4

,one,two,three,four,five,one ret,two ret,four var
0,3.0,8.0,3.0,1.0,nan,nan,nan,nan
1,5.0,16.0,6.0,2.0,nan,1.7,2.0,0.0
2,7.0,28.0,nan,4.0,nan,1.4,1.8,0.7
3,10.0,37.0,nan,7.0,nan,1.4,1.3,1.1
4,13.0,45.0,15.0,11.0,16.0,1.3,1.2,1.4
5,16.0,57.0,18.0,16.0,19.0,1.2,1.3,1.6
6,56.0,69.0,nan,65.0,82.0,3.5,1.2,3.9
7,72.0,90.0,nan,88.0,91.0,1.3,1.3,3.1


## 8 Reindex

`DataFrame.reindex` method conforms a `DataFrame` to a new index, filling cells with no values. It is possible to use this method to rearrange columns.

In [40]:
d5 = d4.reindex(columns=['one','one ret','two','two ret','four','four var'])
d5

,one,one ret,two,two ret,four,four var
0,3.0,nan,8.0,nan,1.0,nan
1,5.0,1.7,16.0,2.0,2.0,0.0
2,7.0,1.4,28.0,1.8,4.0,0.7
3,10.0,1.4,37.0,1.3,7.0,1.1
4,13.0,1.3,45.0,1.2,11.0,1.4
5,16.0,1.2,57.0,1.3,16.0,1.6
6,56.0,3.5,69.0,1.2,65.0,3.9
7,72.0,1.3,90.0,1.3,88.0,3.1


Notice that `DataFrame.reindex` gives a new view, hence `d4` isn't changed.

In [41]:
d4

,one,two,three,four,five,one ret,two ret,four var
0,3.0,8.0,3.0,1.0,nan,nan,nan,nan
1,5.0,16.0,6.0,2.0,nan,1.7,2.0,0.0
2,7.0,28.0,nan,4.0,nan,1.4,1.8,0.7
3,10.0,37.0,nan,7.0,nan,1.4,1.3,1.1
4,13.0,45.0,15.0,11.0,16.0,1.3,1.2,1.4
5,16.0,57.0,18.0,16.0,19.0,1.2,1.3,1.6
6,56.0,69.0,nan,65.0,82.0,3.5,1.2,3.9
7,72.0,90.0,nan,88.0,91.0,1.3,1.3,3.1


## 9 More on Indexing: Multi Index

In [42]:
d6 = pd.read_csv('temp/p01_d4.csv', index_col=['Country',
                                               'Number',
                                               'Dir'])
d6 = d6.sortlevel()
d6

0    1    2
Country Number Dir               
Fra     one    x    0.8 -0.4 -0.4
        two    y    0.9 -0.7  1.0
               z   -0.6  0.5 -2.2
Ger     one    x   -1.5 -0.6  0.7
Jap     one    x    1.0 -1.8  0.4
        two    x   -1.3  1.8  0.6
USA     one    y   -1.5  0.4 -2.3
               z   -0.6  1.1 -0.2

Try by yourself:

    d6.ix['Fra']
    d6.ix['Fra', 'two']
    d6.ix['Fra':'Ger']
    d6.reorder_levels([2,1,0], axis=0).sortlevel(0)
    d6.reset_index(level=1)

In [43]:
d6.ix['Fra'] 

0    1    2
Number Dir               
one    x    0.8 -0.4 -0.4
two    y    0.9 -0.7  1.0
       z   -0.6  0.5 -2.2

## 10 Package Options

The way the DataFrames are displayed can be customized ijn many ways: ([See documentation](http://pandas.pydata.org/pandas-docs/stable/basics.html#working-with-package-options)):

In [44]:
print 'Display Max Rows: \t',  pd.get_option('display.max_rows')
pd.describe_option('display.max_rows')

Display Max Rows: 	60
display.max_rows : int
    If max_rows is exceeded, switch to truncate view. Depending on
    `large_repr`, objects are either centrally truncated or printed as
    a summary view. 'None' value means unlimited.

    In case python/IPython is running in a terminal and `large_repr`
    equals 'truncate' this can be set to 0 and pandas will auto-detect
    the height of the terminal and print a truncated object which fits
    the screen height. The IPython notebook, IPython qtconsole, or
    IDLE do not run in a terminal and hence it is not possible to do
    correct auto-detection.
    [default: 60] [currently: 60]




In [45]:
pd.set_option('display.max_rows', 10)
pd.get_option('display.max_rows')

10

In [46]:
pd.reset_option('display.max_rows')
pd.get_option('display.max_rows')

60

**Try by yourself**:

    pd.describe_option('display.chop_threshold')
    pd.describe_option('display.colheader_justify')
    pd.describe_option('display.column_space')
    pd.describe_option('display.date_dayfirst')
    pd.describe_option('display.date_yearfirst')
    pd.describe_option('display.encoding')
    pd.describe_option('display.expand_frame_repr')
    pd.describe_option('display.float_format')
    pd.describe_option('display.max_columns')
    pd.describe_option('display.max_colwidth')
    pd.describe_option('display.max_rows')
    pd.describe_option('display.notebook_repr_html')
    pd.describe_option('display.precision')
    # for more options see documentation...

In [47]:
pd.set_option('display.precision', 2)
pd.set_option('display.notebook_repr_html', False)
prices

            AAPL  GOOG   JNJ  XOM
Date                             
2015-03-03 129.4 573.6 102.3 87.6
2015-03-04 128.5 573.4 101.7 87.2
2015-03-05 126.4 575.3 102.5 86.7
2015-03-06 126.6 567.7 100.1 85.6
2015-03-09 127.1 568.9 100.7 85.2
2015-03-10 124.5 555.0  99.5 84.3

In [48]:
pd.set_option('display.notebook_repr_html', True)
prices

,AAPL,GOOG,JNJ,XOM
Date,,,,
2015-03-03,129.4,573.6,102.3,87.6
2015-03-04,128.5,573.4,101.7,87.2
2015-03-05,126.4,575.3,102.5,86.7
2015-03-06,126.6,567.7,100.1,85.6
2015-03-09,127.1,568.9,100.7,85.2
2015-03-10,124.5,555.0,99.5,84.3


---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.